In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import Intersection_Inspection as ii
import Intersection_Inspection as ii

%matplotlib inline

In [6]:
df = pd.read_csv('data/Crashes_in_DC_larger.csv', low_memory=False)
df.head()

,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,...,BICYCLISTSIMPAIRED,DRIVERSIMPAIRED,TOTAL_TAXIS,TOTAL_GOVERNMENT,SPEEDING_INVOLVED,NEARESTINTROUTEID,NEARESTINTSTREETNAME,OFFINTERSECTION,INTAPPROACHDIRECTION,LOCERROR
0,NaN,NaN,38010550,26960077,16088739,2016-06-02T04:28:03.000Z,NaN,NaN,NaN,NaN,...,0,0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,38010551,26970671,16095587,2016-06-12T15:57:22.000Z,NaN,NaN,NaN,NaN,...,0,0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,38010552,27144599,16204725,2016-12-03T14:06:24.000Z,NaN,NaN,NaN,NaN,...,0,0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,38010553,26719689,15159240,2015-10-08T01:35:59.000Z,NaN,NaN,NaN,NaN,...,0,0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,38010554,26722499,15160560,2015-10-10T09:21:46.000Z,NaN,NaN,NaN,NaN,...,0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [7]:
df.columns = [x.lower() for x in df.columns]
df.drop(columns=['locerror','todate'], inplace=True)
df.columns

# Fill in some missing x and y values.
df.x = df.longitude
df.y = df.latitude

df.dropna(how='any', axis=0, inplace=True)

# Reset the index in the reduced df so we avoid index out of bounds errors.
df.reset_index(inplace=True, drop=True) # avoid index out of bounds

# Correct an x-variable with the wrong sign (only one instance).
df.loc[df.x > 0]['x'] = df.x * -1

# Convert ward from string to a numeric value
df['ward_number'] = df.ward.str.split(' ', expand=True)[1]
df.ward_number = pd.to_numeric(df.ward_number)

# Clear out remaining null wards
df.loc[df.ward_number.isnull()][['ward','ward_number']]
df.drop(df.loc[df.ward_number.isnull()].index, axis=0, inplace=True)

In [168]:
%%time
def get_nearby_pts(row):
    xrows = df.iloc[((df['x']-abs(row.x)) + (df['y'] - abs(row.y))).abs().argsort()[:50]].index.tolist()
    return xrows
    


CPU times: user 25 s, sys: 3.68 s, total: 28.7 s
Wall time: 25.1 s


0 5000
5000 10000
10000 15000
15000 20000
20000 25000
25000 30000
30000 35000
35000 40000
40000 45000
45000 50000
50000 55000
55000 60000
60000 65000
65000 70000
70000 75000
75000 80000
80000 85000
85000 90000
90000 95000
95000 100000
100000 105000
105000 110000
110000 115000
115000 120000
120000 125000


In [229]:
%%time
# try again to calculate the closest points
def get_nearby_pts2(row):
    xrows = [5,4,3,2,1]
    #print(df.x, row.x, type(df.x), type(row.x))
    #xrows = df.iloc[math.sqrt((df['x']-row.x)**2 + (df['y'] - row.y)**2).argsort()[:10]].index.tolist()
    print(type(df), type(row), df.shape, row.shape)
    #print(df.x, row.x)
    print(row.x)
    #print(math.sqrt((df.x - row.x)**2 + (df.y - row.y)**2))
    return xrows

df['dummy'] = df[:1].apply(get_nearby_pts2, axis=1)
df['dummy']

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> (128153, 50) (50,)
-77.012155
CPU times: user 7.96 ms, sys: 1.82 ms, total: 9.78 ms
Wall time: 7.68 ms


### Calculating nearest points and distances

In [176]:
lower = (step*int(df.shape[0]/step))
upper = lower + df.shape[0] % step

df['closest_pts'] = df[lower:upper].apply(get_nearby_pts, axis=1)

3153

In [186]:
%time df['closest_pts'] = df.apply(get_nearby_pts, axis=1) #27 minutes for all rows

CPU times: user 30.1 ms, sys: 2.95 ms, total: 33.1 ms
Wall time: 30.1 ms
CPU times: user 27min 39s, sys: 3min 41s, total: 31min 21s
Wall time: 27min 27s
CPU times: user 27min 39s, sys: 3min 41s, total: 31min 21s
Wall time: 27min 27s


In [187]:
df.closest_pts

0         [63815, 64030, 64012, 63824, 63868, 64106, 103...
1         [63815, 64030, 63824, 63868, 64012, 64106, 103...
2         [63815, 64030, 63824, 63868, 64012, 64106, 103...
3         [63815, 64030, 63824, 63868, 64012, 64106, 103...
4         [63815, 64030, 64012, 63824, 63868, 64106, 103...
5         [63815, 64030, 64012, 63824, 63868, 64106, 103...
6         [63815, 64030, 63824, 63868, 64012, 64106, 103...
7         [63815, 64030, 64012, 63824, 63868, 64106, 103...
8         [63815, 64030, 63824, 64012, 63868, 64106, 103...
9         [63815, 64030, 64012, 63824, 63868, 64106, 103...
10        [63815, 64030, 63824, 64012, 63868, 64106, 103...
11        [63815, 64030, 64012, 63824, 63868, 64106, 103...
12        [63815, 64030, 64012, 63824, 63868, 64106, 103...
13        [63815, 64030, 64012, 63824, 63868, 64106, 103...
14        [63815, 64030, 63824, 63868, 64012, 64106, 103...
15        [63815, 64030, 64012, 63824, 63868, 64106, 103...
16        [63815, 64030, 64012, 63824, 6

In [188]:
df.to_csv('data/crashes_with_closest_points.csv')

In [202]:
points = df[:1].closest_pts[0]
#xpoints = df[[points]].loc[df.]
# Are these really the closest points??  I think maybe not.  Calc distances...
import math
mydistances = math.sqrt((df.loc[63815].x - df.loc[64030].x)**2 + (df.loc[63815].y - df.loc[64030].y)**2)
mydistances

0.00030489670382041555

In [214]:
%%time
def get_distances(row):
    distances = []
    for point in row.closest_pts:
        dist = math.sqrt((row.x - df.loc[point].x)**2 + (row.y - df.loc[point].y)**2)
        distances.append(dist)
    
    #print(len(distances))
    return distances

df[:1000].apply(get_distances, axis=1)

CPU times: user 17.8 s, sys: 47.2 ms, total: 17.9 s
Wall time: 17.9 s
